## Recurrent Neural Network:
RNN, sırayla gelen verileri işleyen bir modeldir. Her bir adımda bir kelimenin bilgisine önceki adımdan gelen kelimelerin bilgisini ekleyerek sequential
bir şekilde gider. Her bir kelime işlenirken hidden state adı verilen hafıza hücrelerinde önceki ve o anki kelimenin özet bilgilerini tutar.

```python
h_t = f(W_x * x_t + W_h * h_{t + 1} + b)
```

* **h_t:** Şu anki hidden state
* **x_t:** Şu anki kelimenin embedding vektörü
* **h_{t + 1}:** Önceki adımdaki hidden state
* **W_x ve W_h:** Ağırlıklar.
* **b:** Bias
* **f:** Activation function.

**Not:** Başlangıç hidden state'inin (h_0) değeri gelende 0 olarak başlatılır.


Amacımız Premise ve Hypothesis arasındaki ilişkiyi tahmin etmek.

In [1]:
import torch

device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda', index=0)

In [2]:
import pandas as pd

train_path = "data/nli_tr/snli_tr_train.csv"
val_path = "data/nli_tr/snli_tr_validation.csv"
test_path = "data/nli_tr/snli_tr_test.csv"

train_df = pd.read_csv(train_path, sep=',')
val_df = pd.read_csv(val_path, sep=',')
test_df = pd.read_csv(test_path, sep=',')

In [3]:
train_df.columns, val_df.columns, test_df.columns

(Index(['idx', 'premise', 'hypothesis', 'label'], dtype='object'),
 Index(['idx', 'premise', 'hypothesis', 'label'], dtype='object'),
 Index(['idx', 'premise', 'hypothesis', 'label'], dtype='object'))

In [4]:
train_df.shape, val_df.shape, test_df.shape

((550152, 4), (10000, 4), (10000, 4))

In [5]:
train_df.head()

,idx,premise,hypothesis,label
0,0,"Attaki bir kişi, bozuk bir uçağın üzerinden at...",Bir kişi atını yarışma için eğitiyor.,1
1,1,"Attaki bir kişi, bozuk bir uçağın üzerinden at...",Bir kişi bir lokantada omlet sipariş ediyor.,2
2,2,"Attaki bir kişi, bozuk bir uçağın üzerinden at...","Bir kişi açık havada, at üzerinde.",0
3,3,Fotoğraf makinesinde gülümseyen ve sallayan ço...,Ailelerine gülümsüyorlar.,1
4,4,Fotoğraf makinesinde gülümseyen ve sallayan ço...,Burada çocuklar var.,0


In [6]:
# Drop idx column
train_df = train_df.drop(columns=['idx'])
val_df = val_df.drop(columns=['idx'])
test_df = test_df.drop(columns=['idx'])

In [7]:
# Check label type
train_df['label'].value_counts()

label
 0    183416
 2    183187
 1    182764
-1       785
Name: count, dtype: int64

In [8]:
# Hücre 14'ten sonra bu kodu ekleyin:
def fix_labels(df):
    """Label'ları düzelt: -1 -> 0, 0 -> 1, 1 -> 2, 2 -> 3"""
    df = df.copy()
    label_mapping = {-1: 0, 0: 1, 1: 2, 2: 3}
    df['label'] = df['label'].map(label_mapping)
    return df

train_df = fix_labels(train_df)
val_df = fix_labels(val_df)
test_df = fix_labels(test_df)

In [9]:
train_df.head()

,premise,hypothesis,label
0,"Attaki bir kişi, bozuk bir uçağın üzerinden at...",Bir kişi atını yarışma için eğitiyor.,2
1,"Attaki bir kişi, bozuk bir uçağın üzerinden at...",Bir kişi bir lokantada omlet sipariş ediyor.,3
2,"Attaki bir kişi, bozuk bir uçağın üzerinden at...","Bir kişi açık havada, at üzerinde.",1
3,Fotoğraf makinesinde gülümseyen ve sallayan ço...,Ailelerine gülümsüyorlar.,2
4,Fotoğraf makinesinde gülümseyen ve sallayan ço...,Burada çocuklar var.,1


In [10]:
from torch.utils.data import Dataset

class NLIDataset(Dataset):
    def __init__(self, df):
        self.premises = df['premise'].tolist()
        self.hypotheses = df['hypothesis'].tolist()
        self.labels = df['label'].tolist()
    
    def __len__(self):
        return len(self.labels)
    
    def __getitem__(self, idx):
        premise = self.premises[idx]
        hypothesis = self.hypotheses[idx]
        label = self.labels[idx]
        return premise, hypothesis, label

In [11]:
from torch.utils.data import DataLoader

train_dataset = NLIDataset(train_df)
test_dataset = NLIDataset(test_df)
val_dataset = NLIDataset(val_df)


train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

In [12]:
import os
from huggingface_hub import login
from dotenv import load_dotenv

load_dotenv()

hf_token = os.getenv("HF_TOKEN")
login(hf_token)

/home/legacies/anaconda3/envs/torch_env/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Note: Environment variable`HF_TOKEN` is set and is the current active token independently from the token you've just configured.


In [13]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from transformers import AutoModel, AutoTokenizer

In [14]:
class RNNModel(nn.Module):
    def __init__(self, hidden_size=128, num_classes=4, freeze_bert=True):
        super(RNNModel, self).__init__()
        self.bert = AutoModel.from_pretrained('dbmdz/bert-base-turkish-uncased')
        self.tokenizer = AutoTokenizer.from_pretrained('dbmdz/bert-base-turkish-uncased')
        
        embedding_dim = self.bert.config.hidden_size # 768 for BERT base
        
        self.rnn_premise = nn.RNN(input_size=embedding_dim, hidden_size=hidden_size, batch_first=True)
        self.rnn_hypothesis = nn.RNN(input_size=embedding_dim, hidden_size=hidden_size, batch_first=True)
        
        # Concatenate the outputs of both RNNs and classify using a fully connected layer
        self.fc = nn.Linear(hidden_size * 2, num_classes)
        
        if freeze_bert:
            for param in self.bert.parameters():
                param.requires_grad = False
        
    def encode(self, texts):
        inputs = self.tokenizer(
            texts, 
            return_tensors='pt', 
            padding=True, 
            truncation=True, 
            max_length=512
        )
        
        input_ids = inputs['input_ids'].to(device)
        attention_mask = inputs['attention_mask'].to(device)
        
        outputs = self.bert(input_ids=input_ids, attention_mask=attention_mask)
        embeddings = outputs.last_hidden_state
        return embeddings
    
    def forward(self, premise_list, hypothesis_list):
        # Encode premises and hypotheses
        premise_embeddings = self.encode(premise_list)
        hypothesis_embeddings = self.encode(hypothesis_list)
        
        # Pass through RNNs
        rnn_out_premise, _ = self.rnn_premise(premise_embeddings)
        rnn_out_hypothesis, _ = self.rnn_hypothesis(hypothesis_embeddings)
        
        # Take the last hidden state from both RNNs
        last_hidden_premise = rnn_out_premise[:, -1, :]
        last_hidden_hypothesis = rnn_out_hypothesis[:, -1, :]
        
        # Concatenate the outputs
        combined = torch.cat((last_hidden_premise, last_hidden_hypothesis), dim=1)
        
        # Fully connected layer for classification
        logits = self.fc(combined)
        
        return logits

In [15]:
import torch.optim as optim

model = RNNModel(num_classes=4).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=1e-4)

In [16]:
train_loader.dataset[0]  # Example to check if the dataset works

('Attaki bir kişi, bozuk bir uçağın üzerinden atlar.',
 'Bir kişi atını yarışma için eğitiyor.',
 2)

In [ ]:
num_epochs = 10

for epoch in range(num_epochs):
    # --- Training loop ---
    model.train()
    total_train_loss = 0.0
    correct_train = 0
    total_train = 0
    
    for premises, hypotheses, labels in train_loader:
        labels = labels.to(device)
        
        outputs = model(premises, hypotheses)
        loss = criterion(outputs, labels)
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        total_train_loss += loss.item() * labels.size(0)
        
        pred = torch.argmax(outputs, dim=1)
        correct_train += (pred == labels).sum().item()
        total_train += labels.size(0)
    avg_train_loss = total_train_loss / total_train
    train_accuracy = correct_train / total_train
    
    # --- Validation loop ---
    model.eval()
    total_val_loss = 0.0
    correct_val = 0
    total_val = 0
    
    with torch.no_grad():
        for premises, hypotheses, labels in val_loader:
            labels = labels.to(device)
            
            outputs = model(premises, hypotheses)
            loss = criterion(outputs, labels)
            
            total_val_loss += loss.item() * labels.size(0)
            
            pred = torch.argmax(outputs, dim=1)
            correct_val += (pred == labels).sum().item()
            total_val += labels.size(0)
            
    avg_val_loss = total_val_loss / total_val
    val_accuracy = correct_val / total_val
    
    print(f"Epoch {epoch+1}/{num_epochs}, "
          f"Train Loss: {avg_train_loss:.4f}, Train Accuracy: {train_accuracy:.4f}, "
          f"Val Loss: {avg_val_loss:.4f}, Val Accuracy: {val_accuracy:.4f}")
    

KeyboardInterrupt: 

In [ ]:
# --- Testing loop ---
print("=== Testing Evaluation ===")
model.eval()
correct_test = 0
total_test = 0

with torch.no_grad():
    for premises, hypotheses, labels in test_loader:
        labels = labels.to(device)
        
        outputs = model(premises, hypotheses)
        
        pred = torch.argmax(outputs, dim=1)
        correct_test += (pred == labels).sum().item()
        total_test += labels.size(0)
test_accuracy = correct_test / total_test
print(f"Test Accuracy: {test_accuracy:.4f}")
print("=== Testing Evaluation Completed ===")